In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
results_folder = './results/'

In [3]:
data_folder = '../data'

problem_name = 'syn_1000_800_10_50_3'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

## Data preprocessing and parameters initialization

In [4]:
N, n = R.shape
Mf = 2
nu = 3

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
    'rho':np.sqrt(n), #parameters for ll00
    'diam_X':np.sqrt(2),
    'sigma_f':1,                   
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [5]:
func_x = lambda x, Rx=None: pr.portfolio(R, x, Rx=Rx)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad, rho: pr.llo_oracle(x, r, grad,rho)
prox_func = lambda x, L: pr.proj_simplex(x)

## Run FW

In [6]:
run_alpha_policies = ["line_search","icml", "standard"]
#run_alpha_policies = ['new_lloo','lloo']
#run_alpha_policies = ['backtracking']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x, 
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
iter = 1, stepsize = 0.9999999999417923, criterion = 0.999374804507163, upper_bound=0.030278094597530093, lower_bound=-5.158269983947936, real_Gap=5.188548078545466
Convergence achieved!
iter = 149, stepsize = 5.820766091346741e-11, crit = 6.058417612168245e-11, upper_bound=-4.297026343108683, lower_bound=-4.297026343253867, real_Gap=1.4518430901944157e-10
********* Algorithm starts *********
iter = 1, stepsize = 0.49959984730322254, criterion = 0.4992874997595289, upper_bound=0.030278094597530093, lower_bound=-5.158269983947936, real_Gap=5.188548078545466
iter = 1000, stepsize = 0.0005943903381538703, criterion = 0.0005910497664636464, upper_bound=-4.296283994679424, lower_bound=-4.297465244053374, real_Gap=0.0011812493739498109
iter = 2000, stepsize = 0.00027666912400390686, criterion = 0.00029474901522475995, upper_bound=-4.296657698459106, lower_bound=-4.297244159244738, real_Gap=0.0005864607856320703
iter = 3000, stepsize = 0.000223218091529256

## Run SCOPT

In [7]:
x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
        grad_x,
        hess_mult_x,
        hess_mult_vec_x,
        Mf,
        nu,
        prox_func,
        x0,  
        sc_params,                                              
        eps=terminate_tol,                                              
        print_every=1)
    
results[problem_name]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}

Fista err = 9.939e-06; Subiter = 119; subproblem converged!

iter =    1, stepsize = 5.194e-01, rdiff = 5.792e-01 , f = 0.0302781

iter =    2, stepsize = 6.923e-01, rdiff = 2.797e-01 , f = -2.3233

iter =    3, stepsize = 8.807e-01, rdiff = 8.542e-02 , f = -3.71065

Fista err = 9.884e-06; Subiter =  15; subproblem converged!

iter =    4, stepsize = 9.845e-01, rdiff = 9.939e-03 , f = -4.22803

Fista err = 5.447e-06; Subiter =   3; subproblem converged!

iter =    5, stepsize = 9.998e-01, rdiff = 1.299e-04 , f = -4.29596

Fista err = 1.479e-08; Subiter =   2; subproblem converged!

iter =    6, stepsize = 1.000e+00, rdiff = 1.128e-06 , f = -4.29703

Fista err = 2.801e-08; Subiter =   2; subproblem converged!

iter =    7, stepsize = 1.000e+00, rdiff = 5.706e-08 , f = -4.29703

Fista err = 1.050e-10; Subiter =   2; subproblem converged!

iter =    8, stepsize = 1.000e+00, rdiff = 3.481e-09 , f = -4.29703

Fista err = 3.194e-11; Subiter =   2; subproblem converged!

iter =    9, stepsize

In [8]:
with open(os.path.join(results_folder, 'portfolio', problem_name + '.pckl'), 'wb') as f:
    pickle.dump(results, f)